## Todo:
- [ ] Add researcher tools
    - [ ] Search internet (return list of websites)
    - [ ] Read webpage (return the text from one of those websites, maybe add a different llm call here to summarize the article so the tokens don't get stuck in the messages input tokens)
- [ ] Consider which messages actually need to get stored and passed to each new request.  Maybe add a "History" object that contains a brief chat history instead of passing all messages each time?  ie. "User asked for blog about blahblahblah.  Manager asked outliner for outline."


In [3]:
from enum import Enum
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [6]:
# Define the data classes for structured output
class ManagerRouter(str, Enum):
    END = "End"
    OUTLINER = "Outliner"
    WRITER = "Writer"

class EmployeeRouter(str, Enum):
    MANAGER = "Manager"

class ManagerResponse(BaseModel):
    response: str
    route: ManagerRouter

class EmployeeResponse(BaseModel):
    response: str
    route: EmployeeRouter

In [5]:
# Define the system prompts for our various agents
manager_system_prompt = """You work for the world's best tech blog as the manager and editor of the content development team.  You have two employees - an outliner and a writer.  The outliner is responsible for researching and outlining the content, while the writer is responsible for writing the final blog post based on the outline.  The user will ask you to produce a blog on some topic.  

To do so, follow the steps below:

1.  Ask the outliner to research and outline the content.
2.  Review the outline.  If it needs to be adjusted, provide feedback to the outliner.
3.  Once the outline is acceptable, send it to the writer to write the final blog post.
4.  Review the final blog post.  If it needs to be adjusted, provide feedback to the writer.
5.  Once the final blog post is acceptable, send it to the user.

The "route" portion of your response determines who your message will get sent to.  If the route is OUTLINER, direct your response to the outliner employee.  If the route is WRITER, direct your response to the writer employee.  The route END will end the conversation, sending your final response to the user - your final response should be the final blog content in markdown format.
"""

outliner_system_prompt = """You work for the world's best tech blog as a researcher and outliner on the content development team.  You are responsible for researching and outlining the content for the blog post.  Your manager will ask you to research and outline a blog post on some topic.  You should return a concise, bulleted outline of the blog post to your manager.  If the manager provides feedback, make the suggested changes and return the updated outline to the manager."""

writer_system_prompt = """You work for the world's best tech blog as a writer on the content development team.  You are responsible for writing an informative and entertaining blog post based on the provided outline.  Your manager will provide an outline on some topic, and you should turn it into a well-written blog post in markdown format.  If the manager provides feedback, make the suggested changes and return the updated blog post to the manager."""

sysprompts = {
    "Manager": manager_system_prompt,
    "Outliner": outliner_system_prompt,
    "Writer": writer_system_prompt,
}

In [7]:
# Define the model calling function
def call_model_as_role(role: str, messages: list, sysprompts: dict, client):
    messages = [
        {"role": "system", "content": sysprompts[role]},
    ] + messages[1:]
    response_format = ManagerResponse if role == "Manager" else EmployeeResponse
    raw_response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=response_format,
    )
    r = raw_response.choices[0].message.parsed

    messages.append(
        {
            "role": "assistant",
            "content": f"FROM: {role}\n\nTO:  {r.route.value}\n\n{r.response}"
        }
    )
    return r, messages

In [11]:
def generate_blog_post(user_topic: str, client, sysprompts, max_calls=10):
    messages = [
        {"role": "user", 
         "content": f"Write a blog post on {user_topic}"},
    ]

    print("Manager thinking...")
    raw_response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=messages,
        response_format=ManagerResponse,
    )

    r = raw_response.choices[0].message.parsed

    messages.append(
        {
            "role": "assistant",
            "content": f"FROM: Manager\n\nTO:  {r.route.value}\n\n{r.response}"
        }
    )
    calls = 1

    while True:
        if calls >= max_calls:
            print("Max calls reached.")
            break
        match r.route.value:
            case "Manager":
                print("Manager thinking...")
                r, messages = call_model_as_role(
                    "Manager", messages, sysprompts, client
                )
                calls += 1
            case "Outliner":
                print("Outliner thinking...")
                r, messages = call_model_as_role(
                    "Outliner", messages, sysprompts, client
                )
                calls += 1
            case "Writer":
                print("Writer thinking...")
                r, messages = call_model_as_role(
                    "Writer", messages, sysprompts, client
                )
                calls += 1
            case "End":
                return r, messages
            case _:
                print(f"Error: Invalid route {r.route.value} requested.")
                break

In [12]:
resp, msgs = generate_blog_post("The impact of AI on modern technology", client, sysprompts)
print(f"Final blog post:\n{'*'*50}\n\n{resp.response}")

Manager thinking...
Writer thinking...
Manager thinking...
Final blog post:
**************************************************

# The Impact of AI on Modern Technology: Shaping Tomorrow, Today

Artificial Intelligence (AI), once the stuff of science fiction, has become an integral part of modern technology, revolutionizing how we interact with the world around us. It is reshaping industries, enhancing efficiency, and propelling innovation forward at an unprecedented pace. From self-driving cars to intelligent virtual assistants, AI is the driver of a new era where possibilities are limited only by our imagination.

### 1. Transforming Industries

AI is making waves across various sectors, streamlining operations and enhancing productivity:

- **Healthcare**: AI-powered systems assist doctors by analyzing large datasets to predict patient outcomes, personalize treatments, and even discover potential drug interactions. Robotics equipped with AI algorithms also perform intricate surgeries

## Scratch code for internet search

In [13]:
!pip install beautifulsoup4 google

  Obtaining dependency information for google from https://files.pythonhosted.org/packages/ac/35/17c9141c4ae21e9a29a43acdfd848e3e468a810517f862cad07977bf8fe9/google-3.0.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 845.9 kB/s eta 0:00:00 kB/s eta 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [16]:
from googlesearch import search

query = "Coffee"

result = search(query, num=10, stop=10, pause=2)

In [18]:
results = [r for r in result]

In [19]:
results

['https://en.wikipedia.org/wiki/Coffee',
 'https://en.wikipedia.org/wiki/History_of_coffee',
 'https://en.wikipedia.org/wiki/Coffee_bean',
 'https://en.wikipedia.org/wiki/Coffee_preparation',
 'https://en.wikipedia.org/wiki/Coffee_production',
 'https://www.peets.com/',
 'https://www.peets.com/collections/all-coffees',
 'https://www.peets.com/pages/store-locator',
 'https://www.peets.com/pages/menu',
 'https://www.peets.com/pages/coffeepeople']

In [21]:
import urllib.request

page = urllib.request.urlopen(results[0])
print(page.read())

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Coffee - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-featu

In [23]:
# https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python
from bs4 import BeautifulSoup
url = results[5]
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

for script in soup(["script", "style"]):
    script.extract()

text = soup.get_text()

lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk for chunk in chunks if chunk)
print(text)

Peet's Coffee | The Original Craft Coffee, Est. 1966
Click to view our Accessibility Statement
Skip to content
Skip to navigation
Prev Promo
Free Shipping on orders $49+
Shop Now
Next Promo
Prev Promo
Subscribers get Free Shipping & up to 10% Off
Subscribe Now
Next Promo
Peet’s coffee graphic logo
Free Shipping
Coffee
Shop
About Us
Our Stores
Log in
Search
Search
My Account
Store Locator
Toggle Cart
Toggle Menu
Close Search
Search
Search
Popular Searches
Dark Roast Coffee
Decaf Coffee
Espresso Capsules
Featured
Gift Subscription
The Bright Collection
Subscribe & Save
Best Sellers
Exclusives
Decaf Coffee
Shop All Coffee
Roast
Dark Roast
Medium Roast
Light Roast
Brew Format
Coffee Beans
K-Cup® Pods
Espresso Capsules
Cold Brew
Ultra Coffee Concentrate
Shop Now
Shop
Subscribe & Save
Coffee
Offers
Tea
E-Gift Cards
Mugs + Gear
Shop Now
About Us
Coffee for Coffee People
Sourcing with Impact
Recipes + Blog
Why Peet's?
Brew Guides
Peet's History
Discover. Sip. Savor.
Discover. Sip. Savor.
Our S